# Setup

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Loads the image and gets its dimensions
original_image = cv2.imread('test.jpg')
H, W = original_image.shape[:2]

# Converts the image to RGB instead OpenCV's default BGR format for matplotlib
image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

plt.imshow(image_rgb)
plt.axis('off')
plt.show()

# Edge Detection

In [ ]:
# Converts image to grayscale for easier edge detection
image_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

plt.imshow(image_gray, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Thresholds the grayscale image (removing white background) to get a binary image
_, image_thresh = cv2.threshold(image_gray, 240, 255, cv2.THRESH_BINARY_INV)

plt.imshow(image_thresh, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Uses Canny edge detection algorithm on binary image
image_edges = cv2.Canny(image_thresh, 50, 150)

plt.imshow(image_edges, cmap='gray')
plt.axis('off')
plt.show()

# Contour Detection

In [ ]:
# Morphological operations to close gaps in edges and remove noise
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
image_thresh = cv2.morphologyEx(image_thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
image_thresh = cv2.morphologyEx(image_thresh, cv2.MORPH_OPEN, kernel, iterations=1)

plt.imshow(image_thresh, cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# Finds contours from the edges detected
contour, _ = cv2.findContours(image_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contour = sorted(contour, key=cv2.contourArea, reverse=True)

# Makes a copy of the original image to draw contours on
image_contours = original_image.copy()
cv2.drawContours(image_contours, contour, -1, (255, 0, 255), 3)

plt.imshow(cv2.cvtColor(image_contours, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# Corner & Shape Detection

In [ ]:
image_shape = original_image.copy()
# Sets a minimum area threshold to filter out noise (0.001% of image area)
min_area = 0.00001 * H * W

# Iterates through each contour found
for cnt in contour:
    area = cv2.contourArea(cnt)
    if area < min_area: # Filters out small contours that may be noise
        continue

    # Gets the perimeter of the contour
    perimeter = cv2.arcLength(cnt, True)
    # Uses Douglas-Peucker approximation method to approximate the contour to a polygon
    approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)

    # Gets the number of corners in the approximated polygon
    num_corners = len(approx)

    # Calculates the aspect ratio of the bounding rectangle of the contour
    x, y, w, h = cv2.boundingRect(approx)
    aspect_ratio = float(w) / h

    # Determines the shape based on the number of corners and aspect ratio
    shape = "Unidentified"
    if num_corners == 3:
        shape = "Triangle"
    elif num_corners == 4:
        if 0.95 <= aspect_ratio <= 1.05: # Checks aspect ratio to differentiate between square and rectangle
            shape = "Square"
        else:
            shape = "Rectangle"
    elif num_corners > 4:
        shape = "Circle"

    # Draws the contour and shape name on the image
    cv2.drawContours(image_shape, [approx], -1, (255, 0, 255), 2)
    cv2.putText(image_shape, shape, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 2)

plt.imshow(cv2.cvtColor(image_shape, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()